In [1]:
# import requirements 

import requests
import re
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import zipfile
import tempfile

from IPython.display import display
from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor


%matplotlib inline

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
#Read this
#https://support.kobotoolbox.org/synchronous_exports.html

#FETCH KOBO DATA

# Authentication credentials
username = "guilherme_iablonovski"
password = "k0b0Senha"

# API endpoint and export token
# https://kf.kobotoolbox.org/api/v2/assets/aYyrjnacEdnBdZnXaxMcG5/export-settings/esxHYamRwwfcLdZ6JmwnQZ7/data.xlsx
kobo_api_url = "https://kf.kobotoolbox.org/api/v2/assets/aYyrjnacEdnBdZnXaxMcG5/export-settings/esxHYamRwwfcLdZ6JmwnQZ7"

export_token = "f72722c6ffe97db14144325853528a4b7a1c059b"

def getKoboForm(url):
    # Create a session with authentication
    session = requests.Session()
    session.auth = (username, password)

    # Construct the export URL
    kobo_export_url = f"{url}/data.xlsx"#?format=csv&token={export_token}"

    # Fetch data using the authenticated session
    response = session.get(kobo_export_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Load data into a pandas DataFrame
        kobo_data = pd.read_excel(response.content)
        kobo_data.head()
        return kobo_data
    else:
        print("Error:", response.status_code)
        
kobo_data = getKoboForm(kobo_api_url)
kobo_data

C:\Users\joses\AppData\Local\Temp\ipykernel_8384\2828756581.py:30: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  kobo_data = pd.read_excel(response.content)


,This note can be read out loud,Please fill in this form if you represent a non-government organisation requesting fuel for debris removal purposes.,Implementing agency,Please specify,Contact person name,Contact person phone number,Contact person email address,Do you have a MoU with UNOPS?,Location of planned intervention(s),Please specify the municipality (Deir Al-Balah),Please specify the municipality (Gaza),Please specify the municipality (Khan Younis),Please specify the municipality (North Gaza),Please specify the municipality (Rafah),Type of site,"Please, specify",Please describe the site,Scope of operation,Estimated quantity of debris to be removed,Measurement unit,Liters of diesel required per week for debris removal and recycling purposes in North Gaza (North Gaza & Gaza City),"Liters of diesel required per week for debris removal and recycling purposes in South Gaza (Deir Al-Balah, Khan Younis & Rafah)",Comments,Click on the Validate button below to send your response. Plase visit the Gaza DWG Website to find the responses to this form on the dashboards available.,Please contact UNOPS for details on agreeing an MoU for fuel deliveries.,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,__version__,_tags,_index
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deir Al-Balah,Deir al Balah.,NaN,NaN,NaN,NaN,Road,NaN,clean the roads and open the roads,NaN,300,Metric tonnes (t),500.0,NaN,NaN,NaN,NaN,509204309,5e99250e-4ebf-401b-b913-344d9b453967,2025-07-02 13:43:58,NaN,NaN,submitted_via_web,NaN,vrUo2zbbGfNkhDf8V63RcD,NaN,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Khan Younis,NaN,NaN,Khan Yunis.,NaN,NaN,Road,NaN,Removal of debris from the closed Roads to fac...,NaN,90000,Metric tonnes (t),135000.0,NaN,NaN,NaN,NaN,509613833,b3021b66-fc42-441a-bd1f-a8704aadc8d1,2025-07-03 06:33:49,NaN,NaN,submitted_via_web,NaN,vrUo2zbbGfNkhDf8V63RcD,NaN,2
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deir Al-Balah Khan Younis,Deir al Balah.,NaN,Khan Yunis.,NaN,NaN,Road,NaN,All sites in Deir al-Balah and Khan Younis are...,NaN,34500,Metric tonnes (t),38352.0,NaN,NaN,NaN,NaN,554713303,c9d9d7dc-3feb-49fc-9a92-67af1172e501,2025-09-09 07:08:43,NaN,NaN,submitted_via_web,NaN,vn3LwfJj9CHwtUAVPUauxa,NaN,3
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gaza North Gaza,NaN,Gaza.,NaN,Beit Lahiya. Jabalya.,NaN,Road,NaN,NaN,NaN,70000,NaN,NaN,NaN,NaN,NaN,NaN,580205978,98abb206-0760-4eff-add7-9b2f3b662bd1,2025-10-11 15:38:18,NaN,NaN,submitted_via_web,NaN,vaY28MWx3PuyJsMiDtRXix,NaN,4
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gaza Khan Younis North Gaza,NaN,Ash Shati' Camp. Gaza.,Khan Yunis.,Jabalya.,NaN,Road,NaN,NaN,NaN,3000,NaN,NaN,NaN,NaN,NaN,NaN,584623219,110bdf70-e4a5-44ee-9911-91a1bb328b19,2025-10-17 10:32:21,NaN,NaN,submitted_via_web,NaN,v9j7GrtCx8DU3Tgt3p5HED,NaN,5
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deir Al-Balah Gaza Khan Younis,Al Bureij. Al Maghazi. Al Musaddar. An Nuseira...,Gaza.,Al Qarara. Khan Yunis.,NaN,NaN,Road,NaN,Reason for demolition / debris removal: openin...,NaN,9000,Metric tonnes (t),NaN,NaN,NaN,NaN,NaN,584657848,249825ad-e692-4bf5-a76a-5d424acf7e20,2025-10-17 11:17:45,NaN,NaN,submitted_via_web,NaN,v9j7GrtCx8DU3Tgt3p5HED,NaN,6
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deir Al-Balah Gaza Khan Younis,Al Bureij. Al Maghazi. Al Musaddar. Al Qarara....,Gaza.,Al Qarara. Khan Yunis.,NaN,NaN,Road,NaN,1.\topening the closed roads to facilitate the...,NaN,9000,Metric tonnes (t),42000.0,NaN,NaN,NaN,NaN,584680179,7f67fb6a-3ccd-4d7d-841f-e35fdcc5be16,2025-10-17 11:46:53,NaN,NaN,submitted_via_web,NaN,vn3LwfJj9CHwtUAVPUauxa,NaN,7
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gaza Khan Younis North Gaza,NaN,Ash Shati' Camp. Gaza.,Khan Yunis.,Jabalya.,NaN,Road,NaN,NaN,NaN,3000,Cubic meters (m³),NaN,NaN,NaN,NaN,NaN,588789860,d747f52f-d85b-4247-aba2-2c6ad214b784,2025-10-23 05:15:21,NaN,NaN,submitted_via_web,NaN,v9j7GrtCx8DU3Tgt3p5HED,NaN,8
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Khan Younis,NaN,NaN,Khan Yunis.,NaN,NaN,Road,NaN,NaN,NaN,1000,Cubic meters (m³),NaN,NaN,NaN,NaN,NaN,588791570,920d79ec-c687-4567-9c50-8c4

In [4]:
# 1️⃣ Identifica as colunas que começam com "Please specify the municipality"
cols = [c for c in kobo_data.columns if c.startswith("Please specify the municipality")]

# 2️⃣ Cria uma lista para armazenar os registros explodidos
records = []

# 3️⃣ Itera sobre cada linha e coluna dessas
for _, row in kobo_data.iterrows():
    base_data = {col: row[col] for col in kobo_data.columns if col not in cols}  # mantém outras colunas

    for col in cols:
        val = row[col]
        if pd.isna(val):
            continue

        # Extrai o nome entre parênteses
        match = re.search(r'\((.*?)\)', col)
        governo = match.group(1).strip() if match else ''

        # Divide as respostas usando ". " como separador
        municipios = re.split(r'(?<=\.)\s+', str(val).strip())

        # Limpa e forma a nova lista com "Governo_Município"
        for m in municipios:
            m = m.strip().rstrip('.')
            if m:
                new_row = base_data.copy()
                new_row['Gov_Mun'] = f"{governo}_{m}"
                records.append(new_row)

# 4️⃣ Cria o novo DataFrame com as linhas expandidas
kobo_data_exploded = pd.DataFrame(records).reset_index(drop=True)
kobo_data_exploded

,This note can be read out loud,Please fill in this form if you represent a non-government organisation requesting fuel for debris removal purposes.,Implementing agency,Please specify,Contact person name,Contact person phone number,Contact person email address,Do you have a MoU with UNOPS?,Location of planned intervention(s),Type of site,"Please, specify",Please describe the site,Scope of operation,Estimated quantity of debris to be removed,Measurement unit,Liters of diesel required per week for debris removal and recycling purposes in North Gaza (North Gaza & Gaza City),"Liters of diesel required per week for debris removal and recycling purposes in South Gaza (Deir Al-Balah, Khan Younis & Rafah)",Comments,Click on the Validate button below to send your response. Plase visit the Gaza DWG Website to find the responses to this form on the dashboards available.,Please contact UNOPS for details on agreeing an MoU for fuel deliveries.,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,__version__,_tags,_index,Gov_Mun
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deir Al-Balah,Road,NaN,clean the roads and open the roads,NaN,300,Metric tonnes (t),500.0,NaN,NaN,NaN,NaN,509204309,5e99250e-4ebf-401b-b913-344d9b453967,2025-07-02 13:43:58,NaN,NaN,submitted_via_web,NaN,vrUo2zbbGfNkhDf8V63RcD,NaN,1,Deir Al-Balah_Deir al Balah
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Khan Younis,Road,NaN,Removal of debris from the closed Roads to fac...,NaN,90000,Metric tonnes (t),135000.0,NaN,NaN,NaN,NaN,509613833,b3021b66-fc42-441a-bd1f-a8704aadc8d1,2025-07-03 06:33:49,NaN,NaN,submitted_via_web,NaN,vrUo2zbbGfNkhDf8V63RcD,NaN,2,Khan Younis_Khan Yunis
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deir Al-Balah Khan Younis,Road,NaN,All sites in Deir al-Balah and Khan Younis are...,NaN,34500,Metric tonnes (t),38352.0,NaN,NaN,NaN,NaN,554713303,c9d9d7dc-3feb-49fc-9a92-67af1172e501,2025-09-09 07:08:43,NaN,NaN,submitted_via_web,NaN,vn3LwfJj9CHwtUAVPUauxa,NaN,3,Deir Al-Balah_Deir al Balah
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deir Al-Balah Khan Younis,Road,NaN,All sites in Deir al-Balah and Khan Younis are...,NaN,34500,Metric tonnes (t),38352.0,NaN,NaN,NaN,NaN,554713303,c9d9d7dc-3feb-49fc-9a92-67af1172e501,2025-09-09 07:08:43,NaN,NaN,submitted_via_web,NaN,vn3LwfJj9CHwtUAVPUauxa,NaN,3,Khan Younis_Khan Yunis
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gaza North Gaza,Road,NaN,NaN,NaN,70000,NaN,NaN,NaN,NaN,NaN,NaN,580205978,98abb206-0760-4eff-add7-9b2f3b662bd1,2025-10-11 15:38:18,NaN,NaN,submitted_via_web,NaN,vaY28MWx3PuyJsMiDtRXix,NaN,4,Gaza_Gaza
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gaza North Gaza,Road,NaN,NaN,NaN,70000,NaN,NaN,NaN,NaN,NaN,NaN,580205978,98abb206-0760-4eff-add7-9b2f3b662bd1,2025-10-11 15:38:18,NaN,NaN,submitted_via_web,NaN,vaY28MWx3PuyJsMiDtRXix,NaN,4,North Gaza_Beit Lahiya
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gaza North Gaza,Road,NaN,NaN,NaN,70000,NaN,NaN,NaN,NaN,NaN,NaN,580205978,98abb206-0760-4eff-add7-9b2f3b662bd1,2025-10-11 15:38:18,NaN,NaN,submitted_via_web,NaN,vaY28MWx3PuyJsMiDtRXix,NaN,4,North Gaza_Jabalya
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gaza Khan Younis North Gaza,Road,NaN,NaN,NaN,3000,NaN,NaN,NaN,NaN,NaN,NaN,584623219,110bdf70-e4a5-44ee-9911-91a1bb328b19,2025-10-17 10:32:21,NaN,NaN,submitted_via_web,NaN,v9j7GrtCx8DU3Tgt3p5HED,NaN,5,Gaza_Ash Shati' Camp
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gaza Khan Younis North Gaza,Road,NaN,NaN,NaN,3000,NaN,NaN,NaN,NaN,NaN,NaN,584623219,110bdf70-e4a5-44ee-9911-91a1bb328b19,2025-10-17 10:32:21,NaN,NaN,submitted_via_web,NaN,v9j7GrtCx8DU3Tgt3p5HED,NaN,5,Gaza_Gaza
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gaza Khan Younis North Gaza,Road,NaN,NaN,NaN,3000,NaN,NaN,NaN,NaN,NaN,NaN,584623219,110bdf70-e4a5-44ee-9911-91a1bb328b19,2025-10-17 10:32:21,NaN,NaN,submitted_via_web,NaN,v9j7GrtCx8DU3Tgt3p5HED,NaN,5,Khan Younis_Khan Yunis


In [5]:
gdf_municipality = gpd.read_file("C:/Users/joses/Desktop/UNEP_GazaNeighborhoods_Governorate_Municipality_Singleparts_POINTS.gpkg")

In [6]:
print(gdf_municipality.shape)
print(gdf_municipality.crs)
print(gdf_municipality.geom_type.value_counts())

(27, 13)
EPSG:3857
Point    27
Name: count, dtype: int64


In [8]:
gdf_municipality = gdf_municipality.to_crs(epsg=4326)

In [9]:
# 2️⃣ Faz o join com o gdf, mantendo todos os municípios originais
gdf_filtrado = gdf_municipality.merge(kobo_data_exploded, on='Gov_Mun', how='inner')
gdf_filtrado

,OBJECTID,Neighborhood,Municipality,Governorate,Shape__Area,Shape__Length,ArabicName,governate_name,municipality_name,neighborhood_name,Location_gdf,Gov_Mun,geometry,This note can be read out loud,Please fill in this form if you represent a non-government organisation requesting fuel for debris removal purposes.,Implementing agency,Please specify,Contact person name,Contact person phone number,Contact person email address,Do you have a MoU with UNOPS?,Location of planned intervention(s),Type of site,"Please, specify",Please describe the site,Scope of operation,Estimated quantity of debris to be removed,Measurement unit,Liters of diesel required per week for debris removal and recycling purposes in North Gaza (North Gaza & Gaza City),"Liters of diesel required per week for debris removal and recycling purposes in South Gaza (Deir Al-Balah, Khan Younis & Rafah)",Comments,Click on the Validate button below to send your response. Plase visit the Gaza DWG Website to find the responses to this form on the dashboards available.,Please contact UNOPS for details on agreeing an MoU for fuel deliveries.,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,__version__,_tags,_index
0,28,Abu Mgheiseb,Wadi as Salqa,Deir Al-Balah,1.875443e+06,6378.072751,None,deir_al_balah,wadi_as_salqa,abu_mgheiseb,(Deir Al-Balah / Wadi as Salqa)/Abu Mgheiseb,Deir Al-Balah_Wadi as Salqa,POINT (34.3612 31.39191),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deir Al-Balah Gaza Khan Younis,Road,NaN,Reason for demolition / debris removal: openin...,NaN,9000,Metric tonnes (t),NaN,NaN,NaN,NaN,NaN,584657848,249825ad-e692-4bf5-a76a-5d424acf7e20,2025-10-17 11:17:45,NaN,NaN,submitted_via_web,NaN,v9j7GrtCx8DU3Tgt3p5HED,NaN,6
1,28,Abu Mgheiseb,Wadi as Salqa,Deir Al-Balah,1.875443e+06,6378.072751,None,deir_al_balah,wadi_as_salqa,abu_mgheiseb,(Deir Al-Balah / Wadi as Salqa)/Abu Mgheiseb,Deir Al-Balah_Wadi as Salqa,POINT (34.3612 31.39191),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deir Al-Balah Gaza Khan Younis,Road,NaN,1.\topening the closed roads to facilitate the...,NaN,9000,Metric tonnes (t),42000.0,NaN,NaN,NaN,NaN,584680179,7f67fb6a-3ccd-4d7d-841f-e35fdcc5be16,2025-10-17 11:46:53,NaN,NaN,submitted_via_web,NaN,vn3LwfJj9CHwtUAVPUauxa,NaN,7
2,28,Abu Mgheiseb,Wadi as Salqa,Deir Al-Balah,1.875443e+06,6378.072751,None,deir_al_balah,wadi_as_salqa,abu_mgheiseb,(Deir Al-Balah / Wadi as Salqa)/Abu Mgheiseb,Deir Al-Balah_Wadi as Salqa,POINT (34.3612 31.39191),NaN,NaN,UNDP,NaN,Mohammed Abu Mezyed,Chief Technical Specialist-Debris Management,mohammad.mezyad@undp.org,Yes,Deir Al-Balah Gaza Khan Younis,Road Public buildings,NaN,opening the closed roads to facilitate the res...,"Road opening (e.g., pushing debris to the road...",70000,Metric tonnes (t),62500.0,112500.0,"for Gaza we need weekly around : 15,625 liters...",NaN,NaN,594126958,fc69fc56-f83a-42d7-9286-e581112826ad,2025-10-30 06:38:03,NaN,NaN,submitted_via_web,NaN,vvTrAibjDz3pTeScPNavsX,NaN,15
3,34,Al Basateen,Al Musaddar,Deir Al-Balah,1.565272e+06,7815.068901,None,deir_al_balah,al_musaddar,al_basateen,(Deir Al-Balah / Al Musaddar)/Al Basateen,Deir Al-Balah_Al Musaddar,POINT (34.38568 31.41439),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deir Al-Balah Gaza Khan Younis,Road,NaN,Reason for demolition / debris removal: openin...,NaN,9000,Metric tonnes (t),NaN,NaN,NaN,NaN,NaN,584657848,249825ad-e692-4bf5-a76a-5d424acf7e20,2025-10-17 11:17:45,NaN,NaN,submitted_via_web,NaN,v9j7GrtCx8DU3Tgt3p5HED,NaN,6
4,34,Al Basateen,Al Musaddar,Deir Al-Balah,1.565272e+06,7815.068901,None,deir_al_balah,al_musaddar,al_basateen,(Deir Al-Balah / Al Musaddar)/Al Basateen,Deir Al-Balah_Al Musaddar,POINT (34.38568 31.41439),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deir Al-Balah Gaza Khan Younis,Road,NaN,1.\topening the closed roads to facilitate the...,NaN,9000,Metric tonnes (t),42000.0,NaN,NaN,NaN,NaN,584680179,7f67fb6a-3ccd-4d7d-841f-e35fdcc5be16,2025-10-17 11:46:53,NaN,NaN,submitted_via_web,NaN,vn3LwfJj9CHwtUAVPUauxa,NaN,7
5,53,Al Attarta,Beit Lahiya,North Gaza,4.938257e+

In [10]:
print(gdf_municipality.shape)
print(gdf_municipality.crs)
print(gdf_municipality.geom_type.value_counts())

(27, 13)
EPSG:4326
Point    27
Name: count, dtype: int64


In [11]:
# 4️⃣ Reforça que é um GeoDataFrame e mantém o CRS original
gdf_filtrado = gpd.GeoDataFrame(gdf_filtrado, geometry='geometry', crs=gdf_municipality.crs)

print("Geometrias:", gdf_filtrado.geom_type.value_counts())
print("CRS:", gdf_filtrado.crs)

Geometrias: Point    56
Name: count, dtype: int64
CRS: EPSG:4326


In [ ]:
#Handle columns so they are not modified by arcgis when uploaded
def makeArcGISfriendly(df):
    df.columns = df.columns.str.replace(r"[ ]", "_", regex=True)
    df.columns = df.columns.str.replace(r"[.]", "_", regex=True)
    df.columns = df.columns.str.replace(r"[?]", "_", regex=True)
    df.columns = df.columns.str.replace(r"[']", "", regex=True)
    df.columns = df.columns.str.replace(r"[(]", "", regex=True)
    df.columns = df.columns.str.replace(r"[)]", "", regex=True)
    df.columns = df.columns.str.replace(r"[[]", "", regex=True)
    df.columns = df.columns.str.replace(r"[]]", "", regex=True)
    df.columns = df.columns.str.replace(r"[%]", "_", regex=True)
    df.columns = map(str.lower, df.columns)
    # Remove leading underscores
    df = df.rename(columns={
        '_id':'f_id',
        '_uuid':'f_uuid',

    })
    df.columns = df.columns.str.lstrip('_')
    #Truncate and ensure uniqueness
    seen = {}
    final_cols = []
    max_length = 31
    for col in df.columns:
        base = col[:max_length]
        new_col = base
        i = 1
        while new_col in seen:
            suffix = f"_{i}"
            trim_len = max_length - len(suffix)
            new_col = base[:trim_len] + suffix
            i += 1
        seen[new_col] = True
        final_cols.append(new_col)
    
    # Step 4: Apply to DataFrame
    df.columns = final_cols
    return df

gdf_exploded = makeArcGISfriendly(gdf_filtrado)


Index(['objectid', 'neighborhood', 'municipality', 'governorate',
       'shape__area', 'shape__length', 'arabicname', 'governate_name',
       'municipality_name', 'neighborhood_name', 'location_gdf', 'gov_mun',
       'geometry', 'this_note_can_be_read_out_loud',
       'please_fill_in_this_form_if_you', 'implementing_agency',
       'please_specify', 'contact_person_name', 'contact_person_phone_number',
       'contact_person_email_address', 'do_you_have_a_mou_with_unops_',
       'location_of_planned_interventio', 'type_of_site', 'please,_specify',
       'please_describe_the_site', 'scope_of_operation',
       'estimated_quantity_of_debris_to', 'measurement_unit',
       'liters_of_diesel_required_per_w', 'liters_of_diesel_required_per_1',
       'comments', 'click_on_the_validate_button_be',
       'please_contact_unops_for_detail', 'f_id', 'f_uuid', 'submission_time',
       'validation_status', 'notes', 'status', 'submitted_by', 'version__',
       'tags', 'index'],
      dtype

c:\Users\joses\anaconda3\Lib\site-packages\pandas\core\strings\object_array.py:172: FutureWarning: Possible nested set at position 1
  pat = re.compile(pat, flags=flags)


In [13]:
type(gdf_exploded)

geopandas.geodataframe.GeoDataFrame

In [14]:
print(gdf_exploded.columns)

Index(['objectid', 'neighborhood', 'municipality', 'governorate',
       'shape__area', 'shape__length', 'arabicname', 'governate_name',
       'municipality_name', 'neighborhood_name', 'location_gdf', 'gov_mun',
       'geometry', 'this_note_can_be_read_out_loud',
       'please_fill_in_this_form_if_you', 'implementing_agency',
       'please_specify', 'contact_person_name', 'contact_person_phone_number',
       'contact_person_email_address', 'do_you_have_a_mou_with_unops_',
       'location_of_planned_interventio', 'type_of_site', 'please,_specify',
       'please_describe_the_site', 'scope_of_operation',
       'estimated_quantity_of_debris_to', 'measurement_unit',
       'liters_of_diesel_required_per_w', 'liters_of_diesel_required_per_1',
       'comments', 'click_on_the_validate_button_be',
       'please_contact_unops_for_detail', 'f_id', 'f_uuid', 'submission_time',
       'validation_status', 'notes', 'status', 'submitted_by', 'version__',
       'tags', 'index'],
      dtype

In [15]:
print(gdf_exploded.geometry.name if hasattr(gdf_exploded, 'geometry') else "sem geometria")

geometry


In [16]:
def simplifyGeometries(gdf):
    # Reprojeta para CRS em metros (ex: UTM) antes de calcular centróides
    gdf_proj = gdf.to_crs(epsg=3857)
    gdf_proj['geometry'] = gdf_proj.centroid
    gdf_points = gdf_proj.to_crs(epsg=4326)  # volta para WGS84
    return gdf_points

gdf_exploded = simplifyGeometries(gdf_exploded)

In [17]:
seen = set()
unique_flags = []

for _, row in gdf_exploded.iterrows():
    if row['f_id'] in seen:
        unique_flags.append('')
    else:
        unique_flags.append('yes')
        seen.add(row['f_id'])

gdf_exploded['unique_'] = unique_flags

In [18]:
print(gdf_exploded.crs)

EPSG:4326


In [19]:
gdf_exploded['gov_mun'] = gdf_exploded['gov_mun'].astype(str)
gdf_exploded = gdf_exploded.set_crs('EPSG:4326', allow_override=True)  # ajuste se necessário

In [20]:
planning_file = 'DWG_Gaza_FuelRequestResponses_v3'
master_file = 'DWG_Gaza_FuelRequestResponses'
#gdf_exploded.to_file(planning_file+'.gpkg')
#gdf_exploded_mng.to_file(management_file+'.gpkg')
gdf_exploded.to_file(master_file+'.gpkg')

In [21]:
#Connect to the ArcGIS Enterprise portal
gis = GIS('https://wesrmapportal.unep.org/portal/', 'h.partow', 'R&Runit2024')

In [30]:
print(master_file, type(master_file))

DWG_Gaza_FuelRequestResponses <class 'str'>


In [34]:
# 1️⃣ Exportar para GeoJSON (certifique-se que o CRS está certo)
gdf_exploded = gdf_exploded.to_crs(epsg=4326)
gdf_exploded.to_file("DWG_Gaza_FuelRequestResponses_v3.geojson", driver="GeoJSON")

# 2️⃣ Publicar no portal
new_item_properties = {
    "title": "DWG_Gaza_FuelRequestResponses_v3",
    "tags": "fuel, Gaza, planning",
    "type": "GeoJson"   # <- tipo deve ser explicitamente 'GeoJson'
}

new_item = gis.content.add(new_item_properties, data="DWG_Gaza_FuelRequestResponses_v3.geojson")

# 3️⃣ Publicar como Feature Layer
published_layer = new_item.publish()

display(published_layer)

c:\Users\joses\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


<Item title:"DWG_Gaza_FuelRequestResponses_v3" type:Feature Layer Collection owner:h.partow>

In [31]:
# Access the feature-layer through its URL
planning_file = 'DWG_Gaza_FuelRequestResponses_v1'
master_file = 'DWG_Gaza_FuelRequestResponses'


def searchArcgis(keyword):
    if not isinstance(keyword, str):
        raise TypeError(f"Esperado 'str' em 'keyword', recebido {type(keyword)} → {keyword}")
    results = gis.content.search(f'title:\"{keyword}\"', item_type="Feature Layer")
    if not results:
        print("Nenhum resultado encontrado para", keyword)
        return None
    results_sorted = sorted(results, key=lambda x: x.modified, reverse=True)
    return results_sorted[0]

search_planning = searchArcgis(planning_file)
search_master = searchArcgis(master_file)

display(search_master)
search_master[0]

<Item title:"DWG_Gaza_FuelRequestResponses_v1" type:Feature Layer Collection owner:h.partow>

AttributeError: 'int' object has no attribute 'startswith'

In [35]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection, GeoAccessor, FeatureSet
import geopandas as gpd
import pandas as pd

In [36]:
def updateFeature(search_results, gdf_exploded):
    # Obtém o item do Portal
    item = gis.content.get(search_results[0].id)

    # Acessa a Feature Layer Collection e a primeira camada
    flc = FeatureLayerCollection.fromitem(item)
    layer = flc.layers[0]  # Supondo que seja a primeira camada

    # Step 1: Truncar (limpar) feições existentes
    truncate_result = layer.manager.truncate()
    print("Truncate result:", truncate_result)

    # Step 2: Manter apenas colunas compatíveis com o schema da camada
    expected_fields = [f["name"] for f in layer.properties.fields if f["name"] != "fid"]
    gdf_exploded = gdf_exploded[[col for col in gdf_exploded.columns if col in expected_fields or col == "geometry"]]

    # Step 3: Garantir sistema de coordenadas WGS84
    gdf_exploded = gdf_exploded.to_crs(epsg=4326)

    # Step 4: Converter para Spatially Enabled DataFrame
    sedf = GeoAccessor.from_geodataframe(gdf_exploded)

    # Step 5: Converter para FeatureSet
    fs = FeatureSet.from_dataframe(sedf)

    # Step 6: Inserir novas feições
    result = layer.edit_features(adds=fs.features)
    print("Result:", result)

    return result


# Execução da função
updateFeature(search_master, gdf_exploded)

AttributeError: 'int' object has no attribute 'startswith'

In [ ]:
item = gis.content.get(search_master[0].id)

flc = FeatureLayerCollection.fromitem(item)
layer = flc.layers[0]  # Assuming you're working with the first layer


# Step 1: Truncate features
truncate_result = layer.manager.truncate()
print("Truncate result:", truncate_result)

# Load and inspect shapefile
gdf = gdf_exploded

# Keep only matching columns
expected_fields = [f["name"] for f in layer.properties.fields if f["name"] != "fid"]
gdf = gdf[[col for col in gdf.columns if col in expected_fields or col == "geometry"]]

gdf = gdf.to_crs(epsg=4326)

sedf = GeoAccessor.from_geodataframe(gdf)

fs = FeatureSet.from_dataframe(sedf)

result = layer.edit_features(adds=fs.features)
print("Result:", result)

In [ ]:
print(gdf_exploded.crs)